# Data Cleaning for Gender IDEAL

Before committing, please re-run the kernel with clear any output to avoid any merge issues with jupyter and github.

## Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
#import altair as alt

pd.set_option("max_columns", None) # show all cols
pd.set_option('max_colwidth', None) # show full width of showing cols
pd.set_option("expand_frame_repr", False) # print cols side by side as it's supposed to be

Leadership Demographics Category: Scoring
Leadership Weight: 7.5

In [5]:
dfl = pd.read_csv("C:/Users/Admin/Desktop/gender-ideal-main/clean_final/leader_clean_final.csv", index_col=0)
dfl

,Organization,review_status,proportion_local_div,3A,3B,3C,3D,3E,3F,3G,3H,3I,3J,4A,4B,4C,4D,4E,4F,4G,4H,4I,5A,5B,5C,5D,5E,5F,5G,5H,5I,6A,6B,6C,6D,6E,6F,6G,6H,6I,7A,7B,7C,7D,7E,7F,7G,7H,7I,8A,8B,8C,8D,8E,8F,8G,8H,8I,8J,9A,9B,9C,9D,9E,9F,9G,10,benchmark_group,number_of_employees,number_of_years,workforce,9H
0,Company 1A,In progress,Minimal to no disparities (ie - the differential between our workforce and the population across different groups is 5% or less per group),8.0,100.0,13.0,13.0,13.0,26.0,26.0,13.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,100.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,C-Suite,Board/Advisory Bodies,Senior Management,Management,Mentorship Programs,Internal Promotions,NaN,NaN,Finance,Fewer than 100 Employees,5-14 years,We have a mixed workforce of hourly and salaried employees,NaN
1,Company 1B,No,NaN,216.0,38.0,1.0,1.0,6.0,0.0,0.0,1.0,0.0,0.0,10.0,4.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Finance,100-249 Employees,15 or more years,At least 80% of employees are Salaried,NaN
2,Company 1C,No,NaN,30.0,17.0,3.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0,2.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fewer than 100 Employees,Fewer than 5 years,At least 80% of employees are Hourly,NaN
3,Company 1D,No,NaN,35.0,45.0,12.0,6.0,0.0,6.0,0.0,30.0,50.0,0.0,71.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,0.0,75.0,25.0,0.0,0.0,25.0,0.0,0.0,25.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,Board/Advisory Bodies,NaN,NaN,NaN,NaN,NaN,NaN,Technology,Fewer than 100 Employees,5-14 years,At least 80% of employees are Salaried,NaN
4,Company 1E,No,NaN,83.0,70.0,9.0,18.0,4.0,NaN,7.0,NaN,NaN,NaN,76.0,6.0,9.0,6.0,NaN,3.0,NaN,NaN,NaN,75.0,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Internal Promotions,NaN,NaN,NonProfit,Fewer than 100 Employees,5-14 years,At least 80% of employees are Salaried,NaN
5,Company 1F,No,NaN,7.0,43.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Management,NaN,NaN,NaN,NaN,Finance,Fewer than 100 Employees,Fewer than 5 years,At least 80% of employees are Salaried,NaN
6,Company 1G,No,NaN,269.0,40.0,3.0,4.0,6.0,1.0,2.0,NaN,NaN,NaN,41.0,3.0,2.0,6.0,1.0,0.0,NaN,NaN,NaN,30.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,N/A - No targets set,NaN,Fewer than 100 Employees,Fewer than 5 years,At least 80% of employees are Salaried,NaN
7,Company 1H,No,NaN,55.0,90.0,10.0,10.0,5.0,NaN,10.0,0.0,5.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,No,N/A - No targets set,Technology,250-999 Employees,5-14 years,We have a mixed workforce of hourly and salaried employees,NaN
8,Company 1I,No,NaN,57.0,73.0,5.0,3.0,5.0,0.0,2.0,NaN,4.0,0.0,52.0,0.0,0.0,5.0,0.0,0.0,NaN,0.0,0.0,67.0,8.0,0.0,8.0,0.0,0.0,NaN,NaN,0.0,33.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Senior Management,NaN,NaN,NaN,NaN,Nearing goals (66-99% targets met),NonProfit,Fewer than 100 Employees,5-14 y

In [18]:
# Questions 1, 2, 9, and 10 have equal weight of 25% of the Leadership score.
w1 = 0.25
w2 = 0.25
w9 = 0.25
w10 = 0.25
w = [w1, w2, w9, w10]

leader_weight = 7.5

# For each Company row, generate score by weights determined above.
dfl = dfl[['Organization', 'review_status', 'proportion_local_div', '9A', '9B', '9C', '9D', '9E', '9F', '9G', '10']].copy()

# Function to apply to each row:
def compute_score(row):
    
    # Q9:
    # No = 0% 
    # In Progress = 50%
    # Yes = 100%
    # No Sum
    
    q1 = row['review_status']
    if q1 == 'No':
        s1 = 0
    elif q1 == 'In progress':
        s1 = 0.5
    elif q1 == 'Yes':
        s1 = 1
    else:
        s1 = 0
        
    # Q2: 
    # Signficant disparities (>10%) = 33%
    # Modest disparities (<=10%) = 66%
    # Minimal to no disparities (<=5%) = 100%
    # No/N/A = 0%
    # No Sum
    
    q2 = row['proportion_local_div']
    if q2 == """Significant disparities
    (ie - one or more groups that represent
    >10% population not represented in our workforce)""":
        s2 = 0.33
    elif q2 == """Modest disparities
    (ie - the differential between our workforce
    and the population across different groups is 10% or less per group)""":
        s2 = 0.66
    elif q2 == """Minimal to no disparities
    (ie - the differential between our workforce 
    and the population across different groups is 5% or less per group)""":
        s2 = 1
    else: # N/A or No
        s2 = 0  

    # Q9:
    # C-Suite = 50%
    # Board/Advisory Bodies = 50%
    # Management = 50%
    # Mentorship Programs = 50%
    # Promotions = 50%
    # N/A or No = 0%
    # Multi-pick/ Responses sum; Cap at 100%"
    s9 = 0
    if row['9A'] == 'C-Suite':
        s9 += 0.5
    if row['9B'] == 'Board/Advisory Bodies':
        s9 += 0.5
    if row['9C'] == 'Senior Management':
        s9 += 0.5
    if row['9D'] == 'Management':
        s9 += 0.5
    if row['9E'] == 'Mentorship Programs':
        s9 += 0.5
    if row['9F'] == 'Internal Promotions':
        s9 += 0.5
    #9G excluded since all responses can only be No or NaN
    if s9 > 1:
        s9 = 1
        
    # Q10:
    # N/A - No targets set = 0%
    # Behind goals (<66% or fewer targets met) = 50%
    # Nearing goals (66-99% targets met) = 100%
    # Goals met = 100%
    # Goals exceeded = 100%
    # No Sum
    
    q10 = row['10']
    if q2 == 'Behind goals (<66% or fewer targets met)':
        s10 = 0.5
    elif q10 == 'Nearing goals (66-99% targets met)' or q10 == 'Goals met' or q10 == 'Goals exceeded': 
        s10 = 1
    else: #no targets set or NaN
        s10 = 0
    
    s = [s1, s2, s9, s10]
    
    #preliminary score
    return sum(map(lambda a, b: a * b, s, w))


dfl.loc[:,'leader_prelim_score'] = dfl.apply(compute_score, axis = 1)
dfl.loc[:,'leader_weighted_score'] = dfl['leader_prelim_score'] * leader_weight
final_scores = pd.DataFrame(dfl, columns=['Organization', 'leader_weighted_score'])
final_scores

,Organization,leader_weighted_score
0,Company 1A,2.8125
1,Company 1B,0.0000
2,Company 1C,0.0000
3,Company 1D,0.9375
4,Company 1E,0.9375
5,Company 1F,0.9375
6,Company 1G,0.0000
7,Company 1H,0.0000
8,Company 1I,2.8125
9,Company 1J,0.9375


In [19]:
#Write out to csv
dfl.to_csv("C:/Users/Admin/Desktop/gender-ideal-main/scores/leader_scores_clean.csv")